In [1]:
import fastf1
import pandas as pd

fastf1.Cache.enable_cache('/Users/alexforrester/Documents/F1')

In [2]:
def get_rounds_per_year(year):
    schedule = fastf1.get_event_schedule(year, include_testing=False)
    return len(schedule["RoundNumber"])


def driver_mapping_from_session(results):
    driver_lap_time_df = pd.DataFrame(columns=["driver_id", "team_id", "driver_abbreviation", "status"])

    for inx, row in results.iterrows():
        team_id = row["TeamId"]
        driver_id = row["DriverId"]
        driver_abbreviation = row["Abbreviation"]
        status = row["Status"]
        points = row["Points"]
        position_classified = row["ClassifiedPosition"]
        if position_classified not in ["R", "D", "W", "E", "F", "N"]:
            position_classified = float(position_classified)
        if status not in ["Finished", "+1 Lap", "+2 Lap"]:
            position_classified = None

        new_row = pd.DataFrame([[driver_id, team_id, driver_abbreviation, status, points, position_classified]], columns=["driver_id", "team_id", "driver_abbreviation", "status", "points", "position_classified"])
        driver_lap_time_df = pd.concat([driver_lap_time_df, new_row], ignore_index=True)

    return driver_lap_time_df


def get_driver_laps_list(laps) -> list:
    lap_list = []
    for indx, row in laps.iterrows():
        if row["LapTime"] is pd.NaT:
            continue
        if row["IsAccurate"]:
            seconds = row["LapTime"].total_seconds()
            lap_list.append(seconds)
    return lap_list


def filtered_average(laps_list):
    trimmed_list = laps_list[5:-10]
    return sum(trimmed_list) / len(trimmed_list)


In [3]:
def teammate_ratios(driver, driver_index, drivers):
    driver_team = driver["team_id"]
    team_mate_x = drivers[(drivers['team_id'] == driver_team)]
    team_mate = team_mate_x[team_mate_x["driver_abbreviation"] != driver["driver_abbreviation"]]
    teammate_average = team_mate["average_lap"].iloc[0]
    driver_average = driver["average_lap"]
    if not team_mate.empty and driver_average is not None and teammate_average is not None:
        if teammate_average < driver_average:
            drivers.loc[team_mate.index, 'teammate_average_ratio'] = 1.0
            drivers.at[driver_index, 'teammate_average_ratio'] = (teammate_average / driver_average)
        if teammate_average > driver_average:
            drivers.loc[team_mate.index, 'teammate_average_ratio'] = (driver_average / teammate_average)
            drivers.at[driver_index, 'teammate_average_ratio'] = 1.0
    else:
        print("No teammate found for the driver.") 
    



In [4]:
# def car_comparisions(driver, driver_index, drivers):
    # car performance index

# def points_comparison(driver, driver_index, drivers):
    # driver points percentage of constructors points
    # result["Points"]

def finishing_position_comparision(driver, driver_index, drivers):
    driver_team = driver["team_id"]
    team_mate_x = drivers[(drivers['team_id'] == driver_team)]
    team_mate = team_mate_x[team_mate_x["driver_abbreviation"] != driver["driver_abbreviation"]]
    teammate_position = team_mate["position_classified"].iloc[0]
    driver_position = driver["position_classified"]
    non_classified_position = ["R", "D", "W", "E", "F", "N"]

    # dnf logic, default to 10 place difference for driver collision?

    if driver_position not in non_classified_position and teammate_position not in non_classified_position:
        teammate_position = float(teammate_position)
        driver_position = float(driver_position)
        drivers.loc[driver_index, "position_classified_average"] = driver_position - teammate_position
    else:
        drivers.loc[driver_index, "position_classified"] = None
        print("No teammate found for the driver.")


In [5]:
import numpy as np
from pandas import DataFrame
import math

multiyear_combined = DataFrame()
for year in range(2022, 2023):
    driver_rounds_combined_df = DataFrame()
    for race_round in range(1, get_rounds_per_year(year)+1):
        session = fastf1.get_session(year, race_round, 'Race')
        session.load(telemetry=False, laps=True, weather=False)
        results = session.results
        laps = session.laps

        drivers = driver_mapping_from_session(results)

        best_average = float('inf')
        best_third = float('inf')
        best_fastest = float('inf')

        for indx, driver in drivers.iterrows():
            # Add data data to the drivers DF for each round here, points, dnfs etc todo
            result_object = laps.pick_driver(driver["driver_abbreviation"])
            laps_list = get_driver_laps_list(result_object)
            laps_list.sort()
            status = driver.status

            if len(laps_list) > 20:
                fastest = laps_list[0]
                third = laps_list[min(2, len(laps_list)-1)]
                average = filtered_average(laps_list)
                average_rounded_value = round(average, 3)

                if average_rounded_value < best_average:
                    best_average = average_rounded_value
                if third < best_third:
                    best_third = third
                if fastest < best_fastest:
                    best_fastest = fastest

                drivers.at[indx, 'fastest_lap'] = fastest
                drivers.at[indx, 'third_lap'] = third
                drivers.at[indx, 'average_lap'] = average_rounded_value
            else:
                drivers.at[indx, 'fastest_lap'] = None
                drivers.at[indx, 'third_lap'] = None
                drivers.at[indx, 'average_lap'] = None

        for indx, driver in drivers.iterrows():
            # compare data here, teammates and points todo
            driver_fastest = driver["fastest_lap"]
            driver_third = driver["third_lap"]
            driver_average = driver["average_lap"]

            ratio_to_fastest, ratio_to_third, ratio_to_average = math.nan, math.nan, math.nan
            if driver_fastest:
                ratio_to_fastest = best_fastest / driver_fastest
            if driver_fastest:
                ratio_to_third = best_third / driver_third
            if driver_average:
                ratio_to_average = best_average / driver_average

            drivers.at[indx, 'ratio_to_fastest'] = ratio_to_fastest
            drivers.at[indx, 'ratio_to_third'] = ratio_to_third
            drivers.at[indx, 'ratio_to_average'] = ratio_to_average

            teammate_ratios(driver=driver, driver_index=indx, drivers=drivers)
            finishing_position_comparision(driver=driver, driver_index=indx, drivers=drivers)
        display(drivers)
        if driver_rounds_combined_df.empty:
            driver_rounds_combined_df = drivers
        else:
            group_by_dict = {
                'ratio_to_fastest':'mean', 
                'ratio_to_third':'mean',
                'ratio_to_average':'mean',
                'teammate_average_ratio':'mean',
                'fastest_lap':'mean',
                'third_lap':'mean',
                'average_lap':'mean',
                'driver_id':'first',
                'team_id':'first',
                'status':'first',
                'driver_abbreviation':'first',
                'position_classified_average': 'mean',
                'position_classified': 'mean',
                'points': 'sum',
            }
            driver_rounds_combined_df = pd.concat([driver_rounds_combined_df, drivers]).groupby('driver_abbreviation', as_index=False).agg(group_by_dict)

    if multiyear_combined.empty:
        multiyear_combined = driver_rounds_combined_df
    else:
        group_by_dict = {
            'ratio_to_fastest':'mean', 
            'ratio_to_third':'mean',
            'ratio_to_average':'mean',
            'teammate_average_ratio':'mean',
            'driver_id':'first',
            'team_id':'first',
            'driver_abbreviation':'first',
            'position_classified': 'mean',
            'position_classified_average': 'mean',
            'points': 'sum',
        }
        multiyear_combined = pd.concat([multiyear_combined, driver_rounds_combined_df]).groupby('driver_abbreviation', as_index=False).agg(group_by_dict)
display(multiyear_combined)
    



core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.3.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '55', '44', '63', '20', '77', '31', '22', '14', '24', '47', '18', '23', '3', '4', '6', '27', '11', '1', '10']
/var/folders/cq/lhw7m02j7jlb5fy44tj5f6gr0000gn/T/ipykernel_17481/836641098.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclu

,driver_id,team_id,driver_abbreviation,status,points,position_classified,fastest_lap,third_lap,average_lap,ratio_to_fastest,ratio_to_third,ratio_to_average,teammate_average_ratio,position_classified_average
0,leclerc,ferrari,LEC,Finished,26.0,1.0,95.027,95.427,97.516,1.000000,1.000000,1.000000,1.000000,-1.0
1,sainz,ferrari,SAI,Finished,18.0,2.0,95.740,95.880,97.830,0.992553,0.995275,0.996790,0.996790,1.0
2,hamilton,mercedes,HAM,Finished,15.0,3.0,96.228,96.360,98.153,0.987519,0.990318,0.993510,1.000000,-1.0
3,russell,mercedes,RUS,Finished,12.0,4.0,96.302,96.395,98.576,0.986760,0.989958,0.989247,0.995709,1.0
4,kevin_magnussen,haas,MAG,Finished,10.0,5.0,96.623,96.903,98.853,0.983482,0.984768,0.986475,1.000000,-6.0
5,bottas,alfa,BOT,Finished,8.0,6.0,96.599,96.959,99.246,0.983727,0.984200,0.982569,1.000000,-4.0
6,ocon,alpine,OCO,Finished,6.0,7.0,97.110,97.299,99.120,0.978550,0.980760,0.983818,1.000000,-2.0
7,tsunoda,alphatauri,TSU,Finished,4.0,8.0,97.104,97.268,99.232,0.978611,0.981073,0.982707,1.000000,NaN
8,alonso,alpine,ALO,Finished,2.0,9.0,96.733,97.396,99.441,0.982364,0.979784,0.980642,0.996772,2.0
9,zhou,alfa,ZHO,Finished,1.0,10.0,96.685,96.745,99.272,0.982852,0.986377,0.982311,0.999738,4.0


core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.3.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
core        WARNING 	No lap data for driver 22
core        WARNING 	No lap data for driver 47
core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 22)
core        WARNING 	Failed to perform lap accuracy check - all laps marked as inaccurate (driver 47)
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '55', '11', '63', '31', '4',

,driver_id,team_id,driver_abbreviation,status,points,position_classified,fastest_lap,third_lap,average_lap,ratio_to_fastest,ratio_to_third,ratio_to_average,teammate_average_ratio,position_classified_average
0,max_verstappen,red_bull,VER,Finished,25.0,1.0,91.772,91.861,93.363,0.998496,1.000000,0.999829,1.000000,-3.0
1,leclerc,ferrari,LEC,Finished,19.0,2.0,91.634,92.102,93.347,1.000000,0.997383,1.000000,1.000000,-1.0
2,sainz,ferrari,SAI,Finished,15.0,3.0,91.905,92.242,93.440,0.997051,0.995870,0.999005,0.999005,1.0
3,perez,red_bull,PER,Finished,12.0,4.0,92.042,92.093,93.426,0.995567,0.997481,0.999154,0.999326,3.0
4,russell,mercedes,RUS,Finished,10.0,5.0,92.821,93.035,94.238,0.987212,0.987381,0.990545,1.000000,-5.0
5,ocon,alpine,OCO,Finished,8.0,6.0,93.103,93.424,95.097,0.984222,0.983270,0.981598,1.000000,NaN
6,norris,mclaren,NOR,Finished,6.0,7.0,92.753,93.026,95.301,0.987936,0.987477,0.979497,1.000000,NaN
7,gasly,alphatauri,GAS,Finished,4.0,8.0,93.468,93.616,95.333,0.980378,0.981253,0.979168,NaN,NaN
8,kevin_magnussen,haas,MAG,Finished,2.0,9.0,92.779,92.968,94.904,0.987659,0.988093,0.983594,NaN,NaN
9,hamilton,mercedes,HAM,Finished,1.0,10.0,92.997,93.289,94.852,0.985344,0.984693,0.984133,0.993527,5.0


core           INFO 	Loading data for Australian Grand Prix - Race [v3.3.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '11', '63', '44', '4', '3', '31', '77', '10', '23', '24', '18', '47', '20', '22', '6', '14', '1', '5', '55']
/var/folders/cq/lhw7m02j7jlb5fy44tj5f6gr0000gn/T/ipykernel_17481/836641098.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exc

...

core           INFO 	Loading data for São Paulo Grand Prix - Race [v3.3.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['63', '44', '55', '16', '14', '1', '11', '31', '77', '18', '5', '24', '47', '10', '23', '6', '22', '4', '20', '3']
/var/folders/cq/lhw7m02j7jlb5fy44tj5f6gr0000gn/T/ipykernel_17481/836641098.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer excl

,driver_id,team_id,driver_abbreviation,status,points,position_classified,fastest_lap,third_lap,average_lap,ratio_to_fastest,ratio_to_third,ratio_to_average,teammate_average_ratio,position_classified_average
0,russell,mercedes,RUS,Finished,26.0,1.0,73.785,73.881,75.714,1.000000,1.000000,0.999049,0.999049,-1.0
1,hamilton,mercedes,HAM,Finished,18.0,2.0,73.942,73.957,75.642,0.997877,0.998972,1.000000,1.000000,1.0
2,sainz,ferrari,SAI,Finished,15.0,3.0,73.953,74.024,75.682,0.997728,0.998068,0.999471,1.000000,-1.0
3,leclerc,ferrari,LEC,Finished,12.0,4.0,74.253,74.448,75.845,0.993697,0.992384,0.997323,0.997851,1.0
4,alonso,alpine,ALO,Finished,10.0,5.0,74.164,74.430,76.144,0.994890,0.992624,0.993407,1.000000,-3.0
5,max_verstappen,red_bull,VER,Finished,8.0,6.0,74.195,74.278,75.896,0.994474,0.994655,0.996653,1.000000,-1.0
6,perez,red_bull,PER,Finished,6.0,7.0,74.283,74.911,76.064,0.993296,0.986250,0.994452,0.997791,1.0
7,ocon,alpine,OCO,Finished,4.0,8.0,75.093,75.169,76.649,0.982582,0.982865,0.986862,0.993412,3.0
8,bottas,alfa,BOT,Finished,2.0,9.0,75.511,75.587,76.657,0.977142,0.977430,0.986759,1.000000,-3.0
9,stroll,aston_martin,STR,Finished,1.0,10.0,74.831,75.298,76.963,0.986022,0.981181,0.982836,0.997518,-1.0


core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.3.0]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '55', '63', '4', '31', '18', '3', '5', '22', '24', '23', '10', '77', '47', '20', '44', '6', '14']
/var/folders/cq/lhw7m02j7jlb5fy44tj5f6gr0000gn/T/ipykernel_17481/836641098.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer excl

,driver_id,team_id,driver_abbreviation,status,points,position_classified,fastest_lap,third_lap,average_lap,ratio_to_fastest,ratio_to_third,ratio_to_average,teammate_average_ratio,position_classified_average
0,max_verstappen,red_bull,VER,Finished,25.0,1.0,89.392,89.715,90.208,0.988802,0.988553,0.996708,0.996708,-2.0
1,leclerc,ferrari,LEC,Finished,18.0,2.0,89.719,89.736,90.312,0.985198,0.988321,0.995560,0.999147,-2.0
2,perez,red_bull,PER,Finished,15.0,3.0,88.972,89.089,89.911,0.993470,0.995499,1.000000,1.000000,2.0
3,sainz,ferrari,SAI,Finished,12.0,4.0,88.879,89.098,90.235,0.994509,0.995398,0.996409,1.000000,2.0
4,russell,mercedes,RUS,Finished,10.0,5.0,88.836,88.847,90.286,0.994991,0.998210,0.995847,1.000000,NaN
5,norris,mclaren,NOR,Finished,9.0,6.0,88.391,88.688,90.839,1.000000,1.000000,0.989784,1.000000,-3.0
6,ocon,alpine,OCO,Finished,6.0,7.0,89.333,89.379,90.689,0.989455,0.992269,0.991421,1.000000,NaN
7,stroll,aston_martin,STR,Finished,4.0,8.0,89.620,89.805,91.069,0.986287,0.987562,0.987284,1.000000,-2.0
8,ricciardo,mclaren,RIC,Finished,2.0,9.0,90.785,90.856,91.469,0.973630,0.976138,0.982967,0.993112,3.0
9,vettel,aston_martin,VET,Finished,1.0,10.0,90.312,90.614,91.502,0.978729,0.978745,0.982612,0.995268,2.0


,ratio_to_fastest,ratio_to_third,ratio_to_average,teammate_average_ratio,fastest_lap,third_lap,average_lap,driver_id,team_id,status,driver_abbreviation,position_classified_average,position_classified,points
0,0.977237,0.980481,0.979766,0.999995,86.110324,86.325886,87.735605,albon,williams,Finished,ALB,-2.889648,13.324280,4.0
1,0.979016,0.980836,0.982530,0.993334,85.990344,86.340565,87.474229,alonso,alpine,Finished,ALO,-2.384644,5.893433,81.0
2,0.974336,0.977775,0.978833,0.996777,87.319379,87.547767,88.991469,bottas,alfa,Finished,BOT,0.297852,12.668579,47.0
3,0.970054,0.973533,0.977451,1.000000,86.624000,86.748000,87.233000,de_vries,williams,Finished,DEV,-6.000000,9.000000,2.0
4,0.973411,0.977333,0.979830,0.996618,87.156371,87.364945,88.571157,gasly,alphatauri,Power Unit,GAS,1.264160,13.615742,23.0
5,0.987151,0.991200,0.995528,0.998794,86.181993,86.370040,87.355040,hamilton,mercedes,Finished,HAM,-0.481552,2.471331,233.0
6,0.972284,0.972549,0.971202,0.998324,96.006000,96.296500,98.264500,hulkenberg,aston_martin,Finished,HUL,2.000000,14.500000,0.0
7,0.972449,0.975817,0.974448,0.994416,87.153941,87.366921,88.829095,latifi,williams,Finished,LAT,2.944824,15.404297,2.0
8,0.986306,0.990604,0.995034,0.998799,86.264450,86.425010,87.419369,leclerc,ferrari,Finished,LEC,-0.787598,3.109913,291.0
9,0.966232,0.973157,0.973116,0.995919,92.204967,92.470483,93.724897,kevin_magnussen,haas,Finished,MAG,-0.140259,15.605164,21.0
